# Import all the existing libraries and dataset

In [35]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

In [14]:
raw_csv_path = "./Data_Annotations.csv"
converted_csv_path = "./Data_Annotations_Converted.csv"
raw_df = pd.read_csv(csv_path)
converted_df = pd.read_csv(converted_csv_path)

In [17]:
# Print out both the converted contents
converted_df.head()

,Name of file,Indendation,Indendation Consistent?,Whitespace around code to improve readability?,Used == instead of equals,Single/Nondescriptive variable names,Comments,Empty Catch Block,Generic Exceptions,Explicit Initialization,Method Length,Magic Numbers,# of magic numbers,Line Length,Boolean Exp Complexity,"FINAL CODE QUALITY METRIC (1, Very 1, Decent, -1, Very -1)"
0,Snake.java,4,1,1,-1,2,0,-1,-1,0,6,0,0,25,-1,-1
1,TimeTest.java,2,1,0,-1,6,0,-1,-1,1,9,0,0,70,-1,-1
2,HelloWorld.java,9,0,0,-1,1,1,-1,-1,1,6,1,1,40,-1,1
3,NoDisconnectManager.java,2,1,1,-1,0,5,0,0,1,48,0,0,115,1,1
4,TestIntegerNull.java,2,1,1,1,0,0,-1,-1,1,9,0,0,40,1,-1


In [18]:
# Print out the raw contents
raw_df.head()

,Name of file,Indendation,Indendation Consistent?,Whitespace around code to improve readability?,Used == instead of equals,Single/Nondescriptive variable names,Comments,Empty Catch Block,Generic Exceptions,Explicit Initialization,Method Length,Magic Numbers,# of magic numbers,Line Length,Boolean Exp Complexity,"FINAL CODE QUALITY METRIC (Good, Very Good, Decent, Bad, Very Bad)"
0,Snake.java,4,Yes,Yes,NaN,2,0,NaN,NaN,No,6,No,0,25,NaN,Bad
1,TimeTest.java,2,Yes,No,NaN,6,0,NaN,NaN,Yes,9,No,0,70,NaN,Bad
2,HelloWorld.java,9,No,No,NaN,1,1,NaN,NaN,Yes,6,Yes,1,40,NaN,Good
3,NoDisconnectManager.java,2,Yes,Yes,NaN,0,5,No,No,Yes,48,No,0,115,1,Good
4,TestIntegerNull.java,2,Yes,Yes,Yes,0,0,NaN,NaN,Yes,9,No,0,40,1,Bad


# Start the setup for models

In [24]:
features = converted_df.ix[:, 'Indendation':'Boolean Exp Complexity']
features.head()

,Indendation,Indendation Consistent?,Whitespace around code to improve readability?,Used == instead of equals,Single/Nondescriptive variable names,Comments,Empty Catch Block,Generic Exceptions,Explicit Initialization,Method Length,Magic Numbers,# of magic numbers,Line Length,Boolean Exp Complexity
0,4,1,1,-1,2,0,-1,-1,0,6,0,0,25,-1
1,2,1,0,-1,6,0,-1,-1,1,9,0,0,70,-1
2,9,0,0,-1,1,1,-1,-1,1,6,1,1,40,-1
3,2,1,1,-1,0,5,0,0,1,48,0,0,115,1
4,2,1,1,1,0,0,-1,-1,1,9,0,0,40,1


In [26]:
values = converted_df['FINAL CODE QUALITY METRIC (1, Very 1, Decent, -1, Very -1)']
values.head()

0   -1
1   -1
2    1
3    1
4   -1
Name: FINAL CODE QUALITY METRIC (1, Very 1, Decent, -1, Very -1), dtype: int64

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, values, test_size = 0.20, random_state = 13)

# Models

### SVM

In [41]:
from sklearn import svm
from sklearn.metrics import accuracy_score

In [38]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [43]:
y_pred = clf.predict(X_test)

In [44]:
accuracy_score(y_test, y_pred)

0.58333333333333337